In [80]:
import numpy as np
import logging 
import matplotlib.pyplot as plt 
import copy

In [ ]:
# logging.config()

In [ ]:
def ReLU(x,leaky=False,negative_slope=None): # leaky is a flag for leaky relu, negative slope for leaky relu. 
    if leaky and negative_slope!=None:
        try:
            if negative_slope>0:
                raise ValueError("Negative slope accepts integers only.")            
            return np.where(x<0,x*negative_slope,x)
        
        except ValueError as ve:
            print(f"an unexpected error{ve}")
            
        except Exception as e:            
            print(f"An unexpected error {e}")
            raise 
    return np.maximum(0,x)


In [ ]:
def gradient_ReLU(x):
    return np.where(x>0,1,0)

In [54]:
def normal_initialization(fraction,weight_size):
    fract=np.sqrt(2/fraction)
    return np.random.normal(0,fract,size=weight_size)

In [55]:
def uniform_initialization(fraction,weight_size):
    fract= np.sqrt(6/fraction)
    return np.random.uniform(-fract,fract,size=weight_size)


In [56]:
method_map={
        "uniform":uniform_initialization,
        "normal":normal_initialization
    }

In [98]:
def param_initialize(layer_in,layer_out,mode="he",method="uniform",bias=True):
    """ Returns weights and biases initialized as response to input information

    Args:
        layer_in (_type_): _description_
        layer_out (_type_): _description_
        mode (str, optional): _description_. Defaults to "he".
        method (str, optional): _description_. Defaults to "uniform".

    Raises:
        ValueError: _description_
    """
    
    
    
    weights=np.zeros((layer_in,layer_out))
    if bias:
        biases=np.zeros((1,layer_out))
    
    
    try:        
        calcluation=0
        
        if mode.lower()=='random': return np.random.random(size=(layer_in,layer_out))
        
        elif mode.lower()=="he":
            calcluation=1/(np.sqrt(layer_in+layer_out))
            
        elif mode.lower()=="xavier":
            calcluation=1/(np.sqrt(layer_in))
            
        else:
            raise ValueError("Only accepts 'random','he' and 'xavier' string as arguments.")

        weights=method_map[method](calcluation,(layer_in,layer_out))
        
        
       
    except Exception as e:
        print(f"Error occured: {e}")    

    
    return {"weight":weights,"bias":biases}
    

In [107]:
layer=(2,3)
val=param_initialize(2,3,method="normal")
val['weight']


array([[ 6.87669625,  1.39198511,  2.39157925],
       [-1.09132129,  2.83037356, -2.77389239]])

In [ ]:
custom_hidden_layers=[8,4,2,1]
custom_X=np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15]]) # 5 x 3 

In [117]:
def build_network(hidden_layers,X):
    neural_network={}
    new_hidden=hidden_layers.copy()
    new_hidden.insert(0,X.shape[0])
    for layer in range(len(hidden_layers)):
        neural_network[f"Layer {layer+1}"]=param_initialize(new_hidden[layer],new_hidden[layer+1])
    return neural_network
    

In [118]:
network=build_network(hidden_layers=custom_hidden_layers,X=custom_X)
network

{'Layer 1': {'weight': array([[-1.76179768, -0.83370013, -2.13593502, -3.42922043, -0.04516204,
           1.27836336,  1.33701974, -1.72701866],
         [-3.66793718, -1.48357206, -1.2280597 , -0.86653279,  1.81625132,
          -4.09105723,  0.03174214,  4.60474646],
         [-2.16766279, -2.13071686, -3.71745831,  4.18755064, -0.22080371,
           1.73559823, -1.57911789, -3.62266343],
         [ 3.40503795,  3.40406756, -2.29366699,  3.30333894, -4.42813102,
           2.92299669,  0.71928987, -0.87224619],
         [ 4.43097754, -1.17565712,  0.46288468, -0.39060255, -2.8430712 ,
          -0.8699394 ,  1.85802442,  2.20644959]]),
  'bias': array([[0., 0., 0., 0., 0., 0., 0., 0.]])},
 'Layer 2': {'weight': array([[ 3.08097804,  2.21247941,  3.66332341,  1.54067684],
         [ 0.20684473,  0.09225671, -4.24562634, -3.38844032],
         [-0.6374736 , -2.73676665, -0.50303075, -0.43735182],
         [-1.81201372,  3.5070703 , -2.0847472 ,  0.22879365],
         [ 3.29680445,  0

[5, 1, 2, 3]